# Könnnen wir darstellen worüber USA Kandidaten auf Twitter reden?

Im Grunde wollen wir ein bisschen das hier nachbauen https://www.bloomberg.com/graphics/2020-democratic-presidential-candidate-policies/

Agenda:

- Topic Modeling als Inspiration für Themen
- Themen anhand von Keywords anlegen
- Steamgraphs machen


In [ ]:
#!pip install nltk

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", 60)
pd.set_option("display.max_colwidth", 300)

# Tweets besorgen

- Ich habe 2021 39 Tausend Tweets aus Twitter Accounts folgender Demokraten heruntergeladen.  Ich habe [GetOldTweets3](https://github.com/Mottl/GetOldTweets3/) benutzt. 
- 2022 schien es Probleme zu machen. Das ist leider normal bei solchen scrapern. Dieses Jahr funktionierte [twint](https://github.com/twintproject/twint). 
- Dieses Jahr scheint [TWMD](https://github.com/mmpx12/twitter-media-downloader) zu funktionieren. 
Es ist ein lost Battle :) 

In [ ]:
usernames = [
    'joebiden', 'corybooker','petebuttigieg','juliancastro','kamalaharris',
    'amyklobuchar','betoorourke','berniesanders','ewarren','andrewyang',
    'michaelbennet','governorbullock','billdeblasio','johndelaney',
    'tulsigabbard','waynemessam','timryan','joesestak','tomsteyer',
    'marwilliamson','sengillibrand','hickenlooper','jayinslee',
    'sethmoulton','ericswalwell'
]

# Schauen wir mal rein

In [ ]:
# We don't need all of the columns, let's leave out a lot of them
columns = ['username', 'text', 'date']
df = pd.read_csv("data/tweets.csv", usecols=columns)
df.sample(5)

Wie viele haben wir genau?

In [ ]:
df.shape

Wie viele von jedem Kandidaten?

In [ ]:
df.username.value_counts()

# Topics mit Keywords selbst machen

Wir erstellen eine kurze Liste von Wörtern , die mit dem Thema verbunden sind. Unser Ansatz wird etwas umständlich sein, aber er ist ziemlich flexibel für Dinge, die wir in Zukunft vielleicht tun möchten.

In [ ]:
categories = {
    'immigration': ['immigration', 'border', 'wall'],
    'education': ['students', 'education', 'teacher'],
    'foreign_policy': ['foreign policy', 'peace'],
    'climate_change': ['climate', 'emissions', 'carbon'],
    'economy': ['economy', 'tariffs', 'taxes'],
    'military': ['veterans', 'troops', 'war'],
    'jobs': ['jobs', 'unemployment', 'wages'],
    'drugs': ['drugs', 'opioid'],
    'health': ['health', 'insurance', 'medicare'],
    'repro_rights': ['reproductive', 'abortion'],
    'gun_control': ['gun'],
}
categories

Wir werden diese in einen schönen langen Dataframe aus Wörtern und Kategorienamen verwandeln. 
Wir werden auch die Keywords stemmen, damit sie etwas breiter übereinstimmen: `immigrant`, `immigrants`, and `immigration` sollten alle als  `immigr` gestemmt werden.

In [ ]:
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()

dfs = []
for key,values in categories.items():
    for word in values:
        words = {'category': key, 'term': stemmer.stem(word)}
        dfs.append(words)

terms_df = pd.DataFrame(dfs)
terms_df.head()

# Count Vectorizer mit Stemming

Jetzt werden wir unseren eignen Vektorisierer machen: **Wir werden nur Wörter in unserer Liste zählen** und wir werden nur Ja / Nein (1/0) dh. existiert bzw. existiert nicht für jedes der Wörter aufschreiben.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.toarray())
pd.DataFrame(X.toarray(),columns=vectorizer.get_feature_names_out())

In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

def tokenize(text):
    tokens = word_tokenize(text)
    stems = [stemmer.stem(token) for token in tokens]
    return stems

In [ ]:
term_list = list(terms_df["term"])
term_list[0:5]

In [ ]:
vectorizer = CountVectorizer(tokenizer=tokenize,binary=True,vocabulary=term_list) # binary=True macht nur 0/1 # vocabulary= das Vokabular das wir tracken wollen
matrix = vectorizer.fit_transform(df.text)
words_df = pd.DataFrame(matrix.toarray(),columns=vectorizer.get_feature_names_out())
words_df.head()

Von den ersten fünf Tweets scheint nur zeile 3 in eine Kategorie zu passen - er enthält das Wort **teacher**, also werden wir es in die Bildungskategorie (education) aufnehmen.

Wir werden jede Kategorie durchlaufen und dann prüfen, ob einer der Begriffe für diese Kategorie eine "1" enthält. In diesem Fall weisen wir dieser Zeile eine "1" für die Kategorie zu. Wenn nicht, geben wir eine 0.

Zum Beispiel hat Zeile 3 nicht "students" oder "education", sondern "teacher". Als Ergebnis erhält es eine "1" für die Bildungskategorie.

In [ ]:
terms = ['climat', 'emiss', 'carbon']
words_df[terms]#.any(axis=1).astype(int)

# Von stemmed words wieder zurück zu Topics

In [ ]:
# Group the terms by category, then loop through each category
for category_name, rows in terms_df.groupby('category'):
    # Wandle die begriffe für eine Kategorie in eine einfache liste um z.B. ['student', 'educ', 'teacher']
    terms = list(rows['term'])
    print(f"Looking at {category_name} with terms {terms}")

    # words_df[terms] holt alle columns für 'student', 'educ', und 'teacher'
    # .any(axis=1) schaut ob eine davon eine 1 ist , weisst dann True/False zu
    # .astype(int) wandelt True/False in 1/0 um
    # df[category_name] = weist dann den wert df['education'] zu
    df[category_name] = words_df[terms].any(axis=1).astype(int)

In [ ]:
df.head(4)

## Nach Person gruppieren

Nachdem wir nun eine Reihe von Tweets haben, die mit verschiedenen Kategorien gekennzeichnet sind, können wir beginnen, sie zu zählen und zu klassifizieren. Zum Beispiel können wir sehen, wer am meisten über Jobs twittert. 

In [ ]:
df.groupby('username').drugs.sum().sort_values(ascending=False)

# Nach allen Themen gruppieren

Da diese 0 und 1 unsere einzigen numerischen Spalten sind, können wir den Dataframe einfach auffordern, nach Benutzernamen zu gruppieren und jede Kategorie zu addieren.

In [ ]:
#df = df.drop(columns=["topic_id"])

In [ ]:
overall = df.groupby('username').sum()
overall

Das Problem mit dieser Ansicht ist, dass einige Kandidaten viel twittern und einige Kandidaten viel weniger. Wenn wir es grafisch darstellen, gibt es keinen guten Überblick darüber, welche Themen die Kampagnen der Kandidaten wertschätzen.

In [ ]:
%matplotlib inline
ax = overall.plot(kind='bar', stacked=True, figsize=(13,6), width=0.9)

# Move the legend off of the chart
ax.legend(loc=(1.04,0))

Was wir brauchen ist eine Ansicht, die auf Prozentsätzen basiert. Dazu müssen wir jede Spalte durch die Summe der Zählungen in dieser Spalte teilen. Wegen "Pandas-Logik" müssen wir ".div" anstelle von "/" verwenden. Das Teilungszeichen gibt euch möglicherweise keinen Fehler, aber es gibt definitiv falsche Ergebnisse!

In [ ]:
overall_pct = overall.div(overall.sum(axis=1), axis=0)
overall_pct

In [ ]:
ax = overall_pct.plot(kind='bar', stacked=True, figsize=(13,6), width=0.9)

# Move the legend off of the chart
ax.legend(loc=(1.04,0))

# Mit Plotly visualisieren
Matplotlib sieht oft ewas hässlich aus. Lasst uns lieber plotly nehmen, das ist schön interaktiv. 
Wir müssen dafür den Dataframe etwas re-shapen.

In [ ]:
overall_pct

In [ ]:
reshaped = overall_pct.reset_index().melt(id_vars=['username'], var_name='topic', value_name='pct')
reshaped.head(50)

In [ ]:
!pip install plotly

In [ ]:
import plotly.express as px

fig = px.bar(reshaped, x='username', y='pct', color='topic')
fig.show()

# Steamgraphs machen

Streamgraphs sind eine Visualisierungstechnik, die nicht allzu oft verwendet wird, aber ich glaube die Leute mögen sie. Es ist eine Art gestapeltes Flächendiagramm, das vertikal zentriert ist und eine Handvoll Vor- und Nachteile enthält. 

Wir müssen zunächst das Datum typcasten.

In [ ]:
# Convert the date to a datetime, then pull out the week
df['date'] = pd.to_datetime(df.date)
df.head(2)

In [ ]:
df.dtypes

## Resampling

Wir sehen jetzt, dass es ein datetime64 [ns, UTC] ist, was bedeutet, dass wir Dinge tun können, wie den Tag des Jahres oder den Monat oder die Woche herauszuholen! Wir wollen unsere Daten jedoch nach jeder Woche gruppieren.

Das Gruppieren nach Zeit heisst resampling auf und ist recht einfach. Wir werden die Tweets von Kamala Harris herausziehen und sie dann anhand der Datumsspalte in 8-Tage-Chunks resamplen.

In [ ]:
df[df.username == 'BernieSanders']

In [ ]:
# Resample and make it sum every 7 days
harris = df[df.username == 'BernieSanders'].resample('8D', on='date').sum()
harris.head()

Als Grafik

In [ ]:
ax = harris.plot(kind='area', stacked=True)

# Move the legend off of the chart
ax.legend(loc=(1.04,0))

## Steamgraphs mit Matplotlib

So wie bekommen wir Steamgraphs hin, gottseidank hat Matplotlib ein Beispiel

In [ ]:
import matplotlib.pyplot as plt
fig, ax = plt.subplots(figsize=(10,5))

# Plot a stackplot - https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/stackplot_demo.html
ax.stackplot(harris.index, harris.T, baseline='wiggle', labels=harris.columns)

# Move the legend off of the chart
ax.legend(loc=(1.04,0))

Sieht ok aus aber so zackig. Wir können nicht einfach sagen: "Zeichne glatte Linien!" Matplotlib benötigt echte Daten. Es ist sehr zackig, weil die Daten derzeit nur alle acht Tage vorliegen und jedees Mal ein sehr scharfer Sprung zum nächsten sein kann.

### Glätten mit Pandas

Wir tun so, als hätten wir alle zwei Tage Daten.  Zuerst machen wir eine Liste aller Tage, die wir existieren wollen.

In [ ]:
# Make a list of dates between the first and last 
first = harris.index.min()
last = harris.index.max()

# Go between the first and the last in 2-day chunks
frequency = pd.date_range(start=first, end=last, freq='2D')
frequency[:10]

Jetzt werden wir sie unserem Dataframe hinzufügen. Die Daten werden fehlen, wenn wir die neuen Zeilen hinzufügen, da Pandas sicher sein kann was dort hin soll.

In [ ]:
# Reindex our dataframe, adding a bunch of new days, but missing data!
smooth = harris.reindex(frequency)
smooth.head(6)

Füllen wir diese Daten durch Interpolation aus. Wir werden Pandas sagen, dass er eine quadratische Interpolation verwenden soll, um es schön und glatt zu machen.

In [ ]:
# Plan out 2-day chunks between the first and last days
first = harris.index.min()
last = harris.index.max()
frequency = pd.date_range(start=first, end=last, freq='2D')

# Inject the new (empty) rows, then interpolate new data
smoothed = harris.reindex(frequency).interpolate(method='quadratic')
smoothed.head()

# Das Finale
So jetzt probieren wir es nochmal

In [ ]:
fig, ax = plt.subplots(figsize=(10,5))

# Plot a stackplot - https://matplotlib.org/3.1.1/gallery/lines_bars_and_markers/stackplot_demo.html
ax.stackplot(smoothed.index, smoothed.T,
             baseline='wiggle', labels=smoothed.columns)

# Move the legend off of the chart
ax.legend(loc=(1.04,0))

# Set the title
ax.set_title("Bernie twitter topics")

In hübsch

In [ ]:
plt.style.use('ggplot')